In [1]:
import pandas as pd
import numpy as np

import plotly.express as px

from math import radians, sin, cos, sqrt, atan2

from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans, DBSCAN, MeanShift, estimate_bandwidth
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import silhouette_score

In [2]:
df = pd.read_csv("src/fraud test.csv")
df.describe(include="all")

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
count,555719.000000,555719,5.557190e+05,555719,555719,555719.000000,555719,555719,555719,555719,...,555719.000000,555719.000000,5.557190e+05,555719,555719,555719,5.557190e+05,555719.000000,555719.000000,555719.000000
unique,NaN,226976,NaN,693,14,NaN,341,471,2,924,...,NaN,NaN,NaN,478,910,555719,NaN,NaN,NaN,NaN
top,NaN,15/12/2020 21:26,NaN,fraud_Kilback LLC,gas_transport,NaN,Christopher,Smith,F,444 Robert Mews,...,NaN,NaN,NaN,Film/video editor,23/03/1977,2da90c7d74bd46a0caf3777415b3ebd3,NaN,NaN,NaN,NaN
freq,NaN,16,NaN,1859,56370,NaN,11443,12146,304886,1474,...,NaN,NaN,NaN,4119,2408,1,NaN,NaN,NaN,NaN
mean,277859.000000,NaN,4.178387e+17,NaN,NaN,69.392810,NaN,NaN,NaN,NaN,...,38.543253,-90.231325,8.822189e+04,NaN,NaN,NaN,1.380679e+09,38.542798,-90.231380,0.003860
std,160422.401459,NaN,1.309837e+18,NaN,NaN,156.745941,NaN,NaN,NaN,NaN,...,5.061336,13.721780,3.003909e+05,NaN,NaN,NaN,5.201104e+06,5.095829,13.733071,0.062008
min,0.000000,NaN,6.041621e+10,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,...,20.027100,-165.672300,2.300000e+01,NaN,NaN,NaN,1.371817e+09,19.027422,-166.671575,0.000000
25%,138929.500000,NaN,1.800430e+14,NaN,NaN,9.630000,NaN,NaN,NaN,NaN,...,34.668900,-96.798000,7.410000e+02,NaN,NaN,NaN,1.376029e+09,34.755302,-96.905129,0.000000
50%,277859.000000,NaN,3.521420e+15,NaN,NaN,47.290000,NaN,NaN,NaN,NaN,...,39.371600,-87.476900,2.408000e+03,NaN,NaN,NaN,1.380762e+09,39.376593,-87.445204,0.000000
75%,416788.500000,NaN,4.635330e+15,NaN,NaN,83.010000,NaN,NaN,NaN,NaN,...,41.894800,-80.175200,1.968500e+04,NaN,NaN,NaN,1.385867e+09,41.954163,-80.264637,0.000000


In [3]:
df.head(-5)

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,21/06/2020 12:14,2.291160e+15,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,19/03/1968,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,21/06/2020 12:14,3.573030e+15,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",17/01/1990,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,21/06/2020 12:14,3.598220e+15,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",21/10/1970,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,21/06/2020 12:15,3.591920e+15,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,25/07/1987,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,21/06/2020 12:15,3.526830e+15,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,06/07/1955,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555709,555709,31/12/2020 23:57,3.034470e+13,"fraud_Larkin, Stracke and Greenfelder",entertainment,46.71,Christine,Johnson,F,8011 Chapman Tunnel Apt. 568,...,39.8127,-120.6405,1725,Chartered legal executive (England and Wales),27/05/1967,a7105564935ea3977dc61ff9ced3bf5e,1388534238,38.963543,-120.457121,0
555710,555710,31/12/2020 23:57,3.524570e+15,"fraud_Heathcote, Yost and Kertzmann",shopping_net,29.56,Ashley,Cabrera,F,94225 Smith Springs Apt. 617,...,27.6330,-80.4031,105638,"Librarian, public",07/05/1986,9fc9f6f9be3182d519a61a119cf97199,1388534270,27.593881,-80.855092,0
555711,555711,31/12/2020 23:57,3.415460e+14,fraud_Schmidt-Larkin,home,12.68,Mark,Brown,M,8580 Moore Cove,...,64.7556,-165.6723,145,"Administrator, education",09/11/1939,a8310343c189e4a5b6316050d2d6b014,1388534276,65.623593,-165.186033,0
555712,555712,31/12/2020 23:58,5.018030e+11,"fraud_Pouros, Walker and Spencer",kids_pets,13.02,Robert,Flores,M,3277 Fields Meadows Apt. 790,...,41.5403,-122.9366,308,Call centre manager,20/09/1958,bd7071fd5c9510a5594ee196368ac80e,1388534284,41.973127,-123.553032,0


In [4]:
# Séparation des données en deux dataset selon le coté frauduleux de la transaction
df_frauds = df[df['is_fraud'] == 1]
sample = df[df['is_fraud'] == 0]

# Échantillonnage du dataset de transactions non frauduleuses pour avoir la même taille que le dataset de transactions frauduleuses
dsam = sample.sample(2145, random_state=0)

# Fusion des deux datasets
frames = [df_frauds, dsam]
result = pd.concat(frames)

# Mélange des données dans le dataset
result = result.sample(frac = 1)
result.shape

(4290, 23)

In [5]:
result.describe(include="all")

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
count,4290.000000,4290,4.290000e+03,4290,4290,4290.000000,4290,4290,4290,4290,...,4290.000000,4290.000000,4.290000e+03,4290,4290,4290,4.290000e+03,4290.000000,4290.000000,4290.000000
unique,NaN,4237,NaN,680,14,NaN,309,430,2,798,...,NaN,NaN,NaN,445,786,4290,NaN,NaN,NaN,NaN
top,NaN,30/06/2020 23:16,NaN,fraud_Kilback LLC,grocery_pos,NaN,Christopher,Williams,F,444 Robert Mews,...,NaN,NaN,NaN,Science writer,22/09/1997,36a8e917ed023dc99b14309135f32f2d,NaN,NaN,NaN,NaN
freq,NaN,3,NaN,26,693,NaN,119,106,2352,24,...,NaN,NaN,NaN,44,24,1,NaN,NaN,NaN,NaN
mean,261787.030070,NaN,3.810071e+17,NaN,NaN,299.209186,NaN,NaN,NaN,NaN,...,38.783911,-90.376413,8.239483e+04,NaN,NaN,NaN,1.380186e+09,38.784218,-90.380031,0.500000
std,149535.273281,NaN,1.261528e+18,NaN,NaN,374.387217,NaN,NaN,NaN,NaN,...,5.025407,13.864927,2.764307e+05,NaN,NaN,NaN,4.910323e+06,5.052313,13.871188,0.500058
min,370.000000,NaN,6.041621e+10,NaN,NaN,1.020000,NaN,NaN,NaN,NaN,...,20.027100,-165.672300,2.300000e+01,NaN,NaN,NaN,1.371824e+09,19.161782,-166.033127,0.000000
25%,135552.000000,NaN,3.885950e+13,NaN,NaN,20.390000,NaN,NaN,NaN,NaN,...,34.929800,-96.745600,8.610000e+02,NaN,NaN,NaN,1.375913e+09,34.975099,-96.777858,0.000000
50%,259887.000000,NaN,3.500170e+15,NaN,NaN,88.580000,NaN,NaN,NaN,NaN,...,39.599400,-87.823500,2.676000e+03,NaN,NaN,NaN,1.380066e+09,39.589137,-87.703727,0.500000
75%,385890.500000,NaN,4.452370e+15,NaN,NaN,467.127500,NaN,NaN,NaN,NaN,...,42.076500,-80.158000,1.564700e+04,NaN,NaN,NaN,1.384815e+09,42.049737,-80.108471,1.000000


In [6]:
# Définition des colonnes à enlever pour le modèle de clustering
to_drop = ['cc_num', 'first', 'last', 'street', 'city' ,'state', 'zip', 'trans_num']
clean_data = result.drop(columns=to_drop)

In [7]:
# Définition de la fonction de calcul de la distance
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Rayon de la Terre en kilomètres

    # Convertir les latitudes et longitudes en radians
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    # Différences de latitude et longitude
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Formule haversine
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    # Distance en kilomètres
    distance = R * c
    
    return distance
    # Assurez-vous que cette fonction est correctement indentée ici

# Calcul de la distance pour chaque ligne du DataFrame
clean_data['distance'] = clean_data.apply(lambda row: haversine_distance(row['lat'], row['long'], row['merch_lat'], row['merch_long']), axis=1)

# Affichage des premières lignes du DataFrame avec la nouvelle colonne
clean_data.head()

,Unnamed: 0,trans_date_trans_time,merchant,category,amt,gender,lat,long,city_pop,job,dob,unix_time,merch_lat,merch_long,is_fraud,distance
215462,215462,05/09/2020 23:03,"fraud_Gottlieb, Considine and Schultz",shopping_net,953.66,F,33.4816,-86.8590,493806,Operational investment banker,28/03/2000,1378422196,34.149063,-87.536831,1,97.106560
527279,527279,25/12/2020 21:40,fraud_Ankunding LLC,shopping_net,5.03,F,41.0539,-106.0763,100,Public librarian,16/04/1974,1388007639,42.053166,-105.675154,0,116.018466
389352,389352,20/11/2020 22:15,fraud_Volkman Ltd,misc_net,666.71,F,41.3851,-80.1752,606,Energy manager,22/09/1997,1384985708,42.354109,-80.667883,1,115.212707
295584,295584,10/10/2020 23:00,fraud_Rolfson-Kunde,personal_care,21.87,F,33.6401,-116.5567,1661,"Therapist, music",19/09/1988,1381446053,33.191719,-116.946022,1,61.574936
351609,351609,03/11/2020 17:59,"fraud_Swaniawski, Lowe and Robel",shopping_pos,828.44,M,39.4850,-74.8776,825,Licensed conveyancer,06/07/1991,1383501596,39.618559,-75.336649,1,42.066055


In [8]:
clean_data.describe(include="all")

,Unnamed: 0,trans_date_trans_time,merchant,category,amt,gender,lat,long,city_pop,job,dob,unix_time,merch_lat,merch_long,is_fraud,distance
count,4290.000000,4290,4290,4290,4290.000000,4290,4290.000000,4290.000000,4.290000e+03,4290,4290,4.290000e+03,4290.000000,4290.000000,4290.000000,4290.000000
unique,NaN,4237,680,14,NaN,2,NaN,NaN,NaN,445,786,NaN,NaN,NaN,NaN,NaN
top,NaN,30/06/2020 23:16,fraud_Kilback LLC,grocery_pos,NaN,F,NaN,NaN,NaN,Science writer,22/09/1997,NaN,NaN,NaN,NaN,NaN
freq,NaN,3,26,693,NaN,2352,NaN,NaN,NaN,44,24,NaN,NaN,NaN,NaN,NaN
mean,261787.030070,NaN,NaN,NaN,299.209186,NaN,38.783911,-90.376413,8.239483e+04,NaN,NaN,1.380186e+09,38.784218,-90.380031,0.500000,76.416120
std,149535.273281,NaN,NaN,NaN,374.387217,NaN,5.025407,13.864927,2.764307e+05,NaN,NaN,4.910323e+06,5.052313,13.871188,0.500058,29.209701
min,370.000000,NaN,NaN,NaN,1.020000,NaN,20.027100,-165.672300,2.300000e+01,NaN,NaN,1.371824e+09,19.161782,-166.033127,0.000000,1.959344
25%,135552.000000,NaN,NaN,NaN,20.390000,NaN,34.929800,-96.745600,8.610000e+02,NaN,NaN,1.375913e+09,34.975099,-96.777858,0.000000,56.089232
50%,259887.000000,NaN,NaN,NaN,88.580000,NaN,39.599400,-87.823500,2.676000e+03,NaN,NaN,1.380066e+09,39.589137,-87.703727,0.500000,78.884395
75%,385890.500000,NaN,NaN,NaN,467.127500,NaN,42.076500,-80.158000,1.564700e+04,NaN,NaN,1.384815e+09,42.049737,-80.108471,1.000000,99.072109


In [9]:
mean_distance_fraudulent = clean_data.loc[clean_data["is_fraud"] == 1, "distance"].mean()

print("Moyenne de la distance pour les transactions frauduleuses :", mean_distance_fraudulent)

Moyenne de la distance pour les transactions frauduleuses : 76.21390498710234


In [10]:
# Sélection des features nécessaires pour le clustering
data_geo = clean_data[["merch_lat", "merch_long", "is_fraud"]]

In [11]:
# Première répartition des transactions

# Créer une palette de couleurs personnalisée avec le rouge pour les valeurs de is_fraud égales à 1
color_scale = px.colors.diverging.RdYlBu_r

# Modifier la couleur pour les valeurs de is_fraud égales à 1 en rouge
color_scale[0] = 'blue'

# Définir le centre de la carte et le niveau de zoom pour afficher toutes les données
center_longitude = data_geo['merch_long'].mean()  # Utiliser la moyenne des longitudes comme centre
center_latitude = data_geo['merch_lat'].mean()  # Utiliser la moyenne des latitudes comme centre
zoom_level = 2  # Niveau de zoom pour afficher toutes les données

fig = px.scatter_mapbox(
        data_geo,
        lat="merch_lat",
        lon="merch_long",
        color="is_fraud",
        mapbox_style="carto-positron",
        title="Répartition géographiques de l'échantillon des transactions",
        color_continuous_scale=color_scale,
        center=dict(lon=center_longitude, lat=center_latitude), 
        zoom=zoom_level
)

fig.show()

# Standardisation des données

In [12]:
num_feat = [0, 1, 2]
num_trans = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_trans, num_feat)])

X= preprocessor.fit_transform(data_geo)
print(X[0:5, :])

[[-0.91753916  0.2049955   1.        ]
 [ 0.64709547 -1.1027827  -1.        ]
 [ 0.7066678   0.70024858  1.        ]
 [-1.10704752 -1.91541546  1.        ]
 [ 0.16515964  1.08463206  1.        ]]


In [13]:
df_fr = data_geo[data_geo['is_fraud']==1]

# Clustering des transactions frauduleuses
## Kmeans
### Silhouette

In [14]:
from sklearn.metrics import silhouette_score

sil = []
k = []

## Commencez à i=3 pour exclure n_clusters=2
for i in range(2, 31):
    kmeans = KMeans(n_clusters=i, random_state=0, n_init='auto')
    kmeans.fit(df_fr)
    sil.append(silhouette_score(df_fr, kmeans.labels_))
    k.append(i)

# Créer une data frame
cluster_scores = pd.DataFrame(sil)
k_frame = pd.Series(k)

# Créer une figure avec Plotly Express
fig = px.bar(data_frame=cluster_scores, x=k, y=cluster_scores.iloc[:, -1])

# Ajouter un titre et des étiquettes d'axe
fig.update_layout(
    yaxis_title="Silhouette Score",
    xaxis_title="# Clusters",
    title="Silhouette Score par cluster"
)

fig.show()  # Afficher la figure

c:\Users\mouto\anaconda3\envs\jedha\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=9.

c:\Users\mouto\anaconda3\envs\jedha\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=9.

c:\Users\mouto\anaconda3\envs\jedha\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=9.

c:\Users\mouto\anaconda3\envs\jedha\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning:

KMeans is known to have a memory leak on Window

On peut voir grâce à la méthode Silhouette que la répartition en clusters est le plus efficace sur les valeurs 3, 7 et 24 en nombres de clusters

### ELBOW

In [15]:
# Instanciate KMeans with k=3 and initialisation with k-means++
# You should always use k-means++ as it alleviate the problem of local minimum convergence
kmeans = KMeans(n_clusters=10, random_state=0)

# Fit kmeans to our dataset
kmeans.fit(df_fr)
# Let's create a loop that will collect the Within-sum-of-square (wcss) for each value K
# Let's use .inertia_ parameter to get the within sum of square value for each value K
wcss =  []
k = []
for i in range (1,21):
    kmeans = KMeans(n_clusters= i, random_state = 0, n_init = 'auto')
    kmeans.fit(df_fr)
    wcss.append(kmeans.inertia_)
    k.append(i)
# Create DataFrame
wcss_frame = pd.DataFrame(wcss)
k_frame = pd.Series(k)

# Create figure
fig= px.line(
    wcss_frame,
    x=k_frame,
    y=wcss_frame.iloc[:,-1]
)

# Create title and axis labels
fig.update_layout(
    yaxis_title="Inertia",
    xaxis_title="# Clusters",
    title="Inertia per cluster"
)

# Render
#fig.show(renderer="notebook")
fig.show() # if using workspace

c:\Users\mouto\anaconda3\envs\jedha\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

c:\Users\mouto\anaconda3\envs\jedha\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=9.

c:\Users\mouto\anaconda3\envs\jedha\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=9.

c:\Users\mouto\anaconda3\envs\jedha\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than availab

In [16]:
kmeans = KMeans(n_clusters=7, random_state=0)
kmeans.fit(df_fr)

# Prédire les clusters pour chaque échantillon
df_fr['cluster'] = kmeans.labels_

# Définir le centre de la carte et le niveau de zoom pour afficher toutes les données
center_longitude = df_fr['merch_long'].mean()  # Utiliser la moyenne des longitudes comme centre
center_latitude = df_fr['merch_lat'].mean()  # Utiliser la moyenne des latitudes comme centre
zoom_level = 2  # Niveau de zoom pour afficher toutes les données

# Créer une carte interactive avec Plotly Express en spécifiant le centre et le zoom
fig = px.scatter_mapbox(df_fr, lon='merch_long', lat='merch_lat', color='cluster', 
                        mapbox_style="carto-positron", title='Clusters des données géographiques selon la méthode Elbow',
                        center=dict(lon=center_longitude, lat=center_latitude), zoom=zoom_level)

# Afficher la carte interactive
fig.show()

c:\Users\mouto\anaconda3\envs\jedha\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

c:\Users\mouto\anaconda3\envs\jedha\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=9.



C:\Users\mouto\AppData\Local\Temp\ipykernel_5348\1455034139.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



On peut voir grâce à la méthode Elbow, que la répartition en cluster n'est pas optimale et peut s'avérer anarchique. Bien que les zones de villes ne soient découpées entre les deux clusters, cette méthode manque de précision

### DBSCAN

In [17]:
eps = 1
min_samples = 3
p = 1.5

In [18]:
db = DBSCAN(eps=eps, min_samples=min_samples, metric="minkowski", p=p, algorithm='brute')
db.fit(df_fr)
display(np.unique(db.labels_, return_counts=True))

# Créer une copie du DataFrame si nécessaire
df_fr_copy = df_fr.copy()

# Convertir les étiquettes de cluster en int64
cluster_labels = db.labels_.astype('int64')

# Ajouter les labels de cluster à votre DataFrame en utilisant .loc
df_fr_copy.loc[:, 'cluster'] = cluster_labels

#
# Calculer le nombre d'éléments dans chaque cluster
cluster_sizes = df_fr_copy[df_fr_copy['cluster'] != -1]['cluster'].value_counts()

# Trier les clusters par ordre croissant de nombre d'éléments
sorted_clusters = cluster_sizes.sort_values()

# Créer une palette de couleurs divergente
colorscale = px.colors.diverging.balance[::-1]  # Utilisation d'une palette de couleurs divergente

# Mapper chaque cluster à une couleur en fonction de sa taille
cluster_color_mapping = {cluster: i for i, cluster in enumerate(sorted_clusters.index)}

# Ajouter les labels de cluster à votre DataFrame en utilisant .loc
df_fr_copy.loc[:, 'cluster'] = df_fr_copy['cluster'].map(cluster_color_mapping)

# Définir le centre de la carte et le niveau de zoom pour afficher toutes les données
center_longitude = df_fr_copy['merch_long'].mean()  # Utiliser la moyenne des longitudes comme centre
center_latitude = df_fr_copy['merch_lat'].mean()  # Utiliser la moyenne des latitudes comme centre
zoom_level = 2.5  # Niveau de zoom pour afficher toutes les données

# Créer une carte interactive avec Plotly Express en spécifiant le centre et le zoom
fig_fraud = px.scatter_mapbox(df_fr_copy[df_fr_copy['cluster'] != -1], lon='merch_long', lat='merch_lat', color='cluster', 
                        mapbox_style="carto-positron", title='Clusters des zones de transactions frauduleuses selon la méthode DBSCAN',
                        center=dict(lon=center_longitude, lat=center_latitude), zoom=zoom_level,
                        color_continuous_scale=colorscale)

# Afficher la carte interactive
fig_fraud.show()


(array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28], dtype=int64),
 array([ 19, 439, 645,  43, 379,  28,  27,   5, 211,  71,   9,  21,  46,
         26,   8,  36,  22,  16,  14,  14,   9,  12,   9,   7,   7,   5,
          3,   5,   6,   3], dtype=int64))

C:\Users\mouto\AppData\Local\Temp\ipykernel_5348\1882556984.py:12: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0  1  2 ... 20 12  1]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.

C:\Users\mouto\AppData\Local\Temp\ipykernel_5348\1882556984.py:28: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[27. 28. 22. ... 12. 18. 28.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



On peut voir, grâce au dbscan, que les transactions frauduleuses sont réparties principalement autour de grandes villes. Cependant la côte Est est beaucoup plus touchée par le phénomène que la côte Ouest.

### Mean Shift Clustering

In [19]:
# Estimation de la bande passante (bandwidth) automatique
bandwidth = estimate_bandwidth(df_fr, quantile=0.2, n_samples=10)

# Création du modèle Mean Shift
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)

# Entraînement du modèle
ms.fit(df_fr)

# Récupération des labels de cluster et des centres de cluster
labels = ms.labels_
cluster_centers = ms.cluster_centers_

# Nombre de clusters
n_clusters = len(cluster_centers)

print("Nombre de clusters:", n_clusters)

# Ajout des labels de cluster à votre DataFrame
df_fr_copy = df_fr.copy()
df_fr_copy['cluster'] = labels

# Calcul du nombre d'éléments dans chaque cluster
cluster_sizes = df_fr_copy[df_fr_copy['cluster'] != -1]['cluster'].value_counts()

# Tri des clusters par ordre croissant de nombre d'éléments
sorted_clusters = cluster_sizes.sort_values()

# Création d'une palette de couleurs divergente
colorscale = px.colors.diverging.balance[::-1]

# Mappage de chaque cluster à une couleur en fonction de sa taille
cluster_color_mapping = {cluster: i for i, cluster in enumerate(sorted_clusters.index)}

# Ajout des labels de cluster à votre DataFrame
df_fr_copy['cluster'] = df_fr_copy['cluster'].map(cluster_color_mapping)

# Définition du centre de la carte et du niveau de zoom pour afficher toutes les données
center_longitude = df_fr_copy['merch_long'].mean()
center_latitude = df_fr_copy['merch_lat'].mean()
zoom_level = 2.5

# Création d'une carte interactive avec Plotly Express
fig_fraud_ms = px.scatter_mapbox(df_fr_copy[df_fr_copy['cluster'] != -1], lon='merch_long', lat='merch_lat', color='cluster', 
                        mapbox_style="carto-positron", title='Clusters des zones de transactions frauduleuses selon la méthode Mean Shift',
                        center=dict(lon=center_longitude, lat=center_latitude), zoom=zoom_level,
                        color_continuous_scale=colorscale)

# Affichage de la carte interactive
fig_fraud_ms.show()


Nombre de clusters: 15


On peut remarquer que la méthode MSC créée les clusters de manières arbitraire. Cette méthode n'est donc pas retenue pour les résultats finaux

# Clustering avec l'ensemble des données

Maintenant que les clusters des transactions frauduleuses sont faites, nous nous attaquons aux données globables afin de pouvoir définir si les transactions frauduleuses suivent le schéma des transactions globales

In [20]:
data_geo.head()

,merch_lat,merch_long,is_fraud
215462,34.149063,-87.536831,1
527279,42.053166,-105.675154,0
389352,42.354109,-80.667883,1
295584,33.191719,-116.946022,1
351609,39.618559,-75.336649,1


### KMeans

In [21]:
from sklearn.metrics import silhouette_score

sil = []
k = []

## Commencez à i=3 pour exclure n_clusters=2
for i in range(3, 31):
    kmeans = KMeans(n_clusters=i, random_state=0, n_init='auto')
    kmeans.fit(data_geo)
    sil.append(silhouette_score(data_geo, kmeans.labels_))
    k.append(i)

# Créer une data frame
cluster_scores = pd.DataFrame(sil)
k_frame = pd.Series(k)

# Créer une figure avec Plotly Express
fig = px.bar(data_frame=cluster_scores, x=k, y=cluster_scores.iloc[:, -1])

# Ajouter un titre et des étiquettes d'axe
fig.update_layout(
    yaxis_title="Silhouette Score",
    xaxis_title="# Clusters",
    title="Silhouette Score par cluster"
)

fig.show()  # Afficher la figure

In [22]:
# Instanciate KMeans with k=3 and initialisation with k-means++
# You should always use k-means++ as it alleviate the problem of local minimum convergence
kmeans = KMeans(n_clusters=10, random_state=0)

# Fit kmeans to our dataset
kmeans.fit(data_geo)
# Let's create a loop that will collect the Within-sum-of-square (wcss) for each value K
# Let's use .inertia_ parameter to get the within sum of square value for each value K
wcss =  []
k = []
for i in range (1,21):
    kmeans = KMeans(n_clusters= i, random_state = 0, n_init = 'auto')
    kmeans.fit(data_geo)
    wcss.append(kmeans.inertia_)
    k.append(i)
# Create DataFrame
wcss_frame = pd.DataFrame(wcss)
k_frame = pd.Series(k)

# Create figure
fig= px.line(
    wcss_frame,
    x=k_frame,
    y=wcss_frame.iloc[:,-1]
)

# Create title and axis labels
fig.update_layout(
    yaxis_title="Inertia",
    xaxis_title="# Clusters",
    title="Inertia per cluster"
)

# Render
#fig.show(renderer="notebook")
fig.show() # if using workspace

c:\Users\mouto\anaconda3\envs\jedha\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [23]:
# Créer une copie du DataFrame si nécessaire
data_geo_copy = data_geo.copy()

# Créer une instance de KMeans et l'ajuster aux données
kmeans = KMeans(n_clusters=5, random_state=0)
kmeans.fit(data_geo_copy)

# Prédire les clusters pour chaque échantillon
data_geo_copy.loc[:, 'cluster'] = kmeans.labels_

# Définir le centre de la carte et le niveau de zoom pour afficher toutes les données
center_longitude = data_geo_copy['merch_long'].mean()  # Utiliser la moyenne des longitudes comme centre
center_latitude = data_geo_copy['merch_lat'].mean()  # Utiliser la moyenne des latitudes comme centre
zoom_level = 2  # Niveau de zoom pour afficher toutes les données

# Créer une carte interactive avec Plotly Express en spécifiant le centre et le zoom
fig = px.scatter_mapbox(data_geo_copy, lon='merch_long', lat='merch_lat', color='cluster', 
                        mapbox_style="carto-positron", title='Clusters des données géographiques selon la méthode KMeans',
                        center=dict(lon=center_longitude, lat=center_latitude), zoom=zoom_level)

# Afficher la carte interactive
fig.show()


c:\Users\mouto\anaconda3\envs\jedha\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



### DBSCAN

In [24]:
eps = 0.8
min_samples = 3
p = 1.6

In [25]:
db = DBSCAN(eps=eps, min_samples=min_samples, metric="minkowski", p=p, algorithm='brute')
db.fit(data_geo)
display(np.unique(db.labels_, return_counts=True))

# Créer une copie du DataFrame si nécessaire
data_geo_copy = data_geo.copy()

# Convertir les étiquettes de cluster en int64
cluster_labels = db.labels_.astype('int64')

# Ajouter les labels de cluster à votre DataFrame en utilisant .loc
data_geo_copy.loc[:, 'cluster'] = cluster_labels

#
# Filtrer les données pour ne conserver que les points assignés à un cluster
data_geo_assigned = data_geo_copy[data_geo_copy['cluster'] != -1]

# Définir le centre de la carte et le niveau de zoom pour afficher toutes les données
center_longitude = data_geo_assigned['merch_long'].mean()
center_latitude = data_geo_assigned['merch_lat'].mean()
zoom_level = 2.5

# Spécifier une palette de couleurs qualitative avec plus de variations
colorscale = px.colors.qualitative.Dark24

# Créer une carte interactive avec Plotly Express en spécifiant le centre et le zoom
fig_all2 = px.scatter_mapbox(data_geo_assigned, lon='merch_long', lat='merch_lat', color='cluster', 
                        mapbox_style="carto-positron", title='Clusters des zones de transactions avec DBSCAN',
                        center=dict(lon=center_longitude, lat=center_latitude), zoom=zoom_level,
                        color_continuous_scale=colorscale)

# Afficher la carte interactive
fig_all2.show()

# Calculer le nombre total de fraudes par cluster
fraud_count = data_geo_assigned.groupby('cluster')['is_fraud'].sum()

# Calculer le pourcentage de fraudes pour chaque cluster
fraud_percentage = data_geo_assigned.groupby('cluster')['is_fraud'].mean() * 100

# Créer un DataFrame à partir des résultats pour faciliter la visualisation avec Plotly Express
fraud_data = pd.DataFrame({'Cluster': fraud_count.index, 
                           'Nombre de fraudes': fraud_count.values,
                           'Pourcentage de fraudes': fraud_percentage.values})

# Trier les données par nombre de fraudes décroissant
fraud_data = fraud_data.sort_values(by='Nombre de fraudes', ascending=False)

# Créer le graphique à barres avec Plotly Express et spécifier la couleur en fonction du nombre de fraudes
fig = px.bar(fraud_data, x='Cluster', y='Pourcentage de fraudes', 
             title='Pourcentage de fraudes par cluster',
             labels={'Pourcentage de fraudes': 'Pourcentage de fraudes (%)'},
             color='Nombre de fraudes')

# Afficher le graphique
fig.show()

(array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32,
        33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
        50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66,
        67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78], dtype=int64),
 array([ 178, 1504,    4,   43, 1471,   28,   27,    4,   25,   35,   71,
           3,   10,   18,  170,    9,   21,   46,   21,   25,   12,   20,
          18,   94,   34,    3,   32,   36,   22,   11,   14,    3,   27,
           6,    9,    9,    9,    6,    6,    6,    7,    9,    8,    8,
           6,    6,    4,    7,    4,    4,    4,    4,    6,    9,   13,
           7,    6,    3,    7,    4,    3,    7,    5,    5,    3,    3,
           4,    3,    3,    6,    3,    4,    3,    3,    3,    4,    3,
           3,    3,    3], dtype=int64))

In [26]:
import plotly.express as px

# Filtrer les clusters avec au moins une fraude
clusters_with_frauds = fraud_data[fraud_data['Nombre de fraudes'] > 0]
fraudulent_clusters = clusters_with_frauds['Cluster'].unique()

# Filtrer les clusters sans fraude
clusters_without_frauds = fraud_data[fraud_data['Nombre de fraudes'] == 0]
non_fraudulent_clusters = clusters_without_frauds['Cluster'].unique()

# Créer une carte interactive pour les clusters avec au moins une fraude
fig_fraudulent_clusters = px.scatter_mapbox(data_geo_assigned[data_geo_assigned['cluster'].isin(fraudulent_clusters)], 
                                            lon='merch_long', lat='merch_lat', color='cluster', 
                                            mapbox_style="carto-positron", 
                                            title='Clusters avec au moins une fraude',
                                            center=dict(lon=center_longitude, lat=center_latitude), 
                                            zoom=zoom_level,
                                            color_continuous_scale=colorscale)
fig_fraudulent_clusters.show()

# Créer une carte interactive pour les clusters sans fraude
fig_non_fraudulent_clusters = px.scatter_mapbox(data_geo_assigned[data_geo_assigned['cluster'].isin(non_fraudulent_clusters)], 
                                                lon='merch_long', lat='merch_lat', color='cluster', 
                                                mapbox_style="carto-positron", 
                                                title='Clusters sans fraude',
                                                center=dict(lon=center_longitude, lat=center_latitude), 
                                                zoom=zoom_level,
                                                color_continuous_scale=colorscale)
fig_non_fraudulent_clusters.show()


analyser les clusters en fonction de is_fraud == 1

# Mean Shift Clustering

In [27]:
from sklearn.cluster import MeanShift, estimate_bandwidth
import numpy as np
import pandas as pd
import plotly.express as px

# Estimation de la bande passante (bandwidth) automatique
bandwidth = estimate_bandwidth(data_geo, quantile=0.19, n_samples=12)

# Création du modèle Mean Shift
ms = MeanShift(bandwidth=bandwidth, bin_seeding=False)

# Entraînement du modèle
ms.fit(data_geo)

# Récupération des labels de cluster et des centres de cluster
labels = ms.labels_
cluster_centers = ms.cluster_centers_

# Nombre de clusters
n_clusters = len(cluster_centers)

print("Nombre de clusters:", n_clusters)

# Ajout des labels de cluster à votre DataFrame
data_geo_copy_msc = data_geo.copy()
data_geo_copy_msc['cluster'] = labels

# Calcul du nombre d'éléments dans chaque cluster
cluster_sizes = data_geo_copy_msc[data_geo_copy_msc['cluster'] != -1]['cluster'].value_counts()

# Tri des clusters par ordre croissant de nombre d'éléments
sorted_clusters = cluster_sizes.sort_values()

# Création d'une palette de couleurs divergente
colorscale = px.colors.diverging.balance[::-1]

# Mappage de chaque cluster à une couleur en fonction de sa taille
cluster_color_mapping = {cluster: i for i, cluster in enumerate(sorted_clusters.index)}

# Ajout des labels de cluster à votre DataFrame
data_geo_copy_msc['cluster'] = data_geo_copy_msc['cluster'].map(cluster_color_mapping)

# Définition du centre de la carte et du niveau de zoom pour afficher toutes les données
center_longitude = data_geo_copy_msc['merch_long'].mean()
center_latitude = data_geo_copy_msc['merch_lat'].mean()
zoom_level = 2.5

# Création d'une carte interactive avec Plotly Express
fig_fraud_ms = px.scatter_mapbox(data_geo_copy_msc[data_geo_copy_msc['cluster'] != -1], lon='merch_long', lat='merch_lat', color='cluster', 
                        mapbox_style="carto-positron", title='Clusters des zones de transactions selon la méthode Mean Shift',
                        center=dict(lon=center_longitude, lat=center_latitude), zoom=zoom_level,
                        color_continuous_scale=colorscale)

# Affichage de la carte interactive
fig_fraud_ms.show()


Nombre de clusters: 18


### DBSCAN sur l'ensemble des données en aveugle (sans la feature is_fraud)

In [28]:
data_geo_blind = data_geo.drop(columns="is_fraud")
data_geo_blind.head()

,merch_lat,merch_long
215462,34.149063,-87.536831
527279,42.053166,-105.675154
389352,42.354109,-80.667883
295584,33.191719,-116.946022
351609,39.618559,-75.336649


In [29]:
eps = 0.9
min_samples = 3
p = 1.6

In [30]:
db = DBSCAN(eps=eps, min_samples=min_samples, metric="minkowski", p=p, algorithm='brute')
db.fit(data_geo_blind)
display(np.unique(db.labels_, return_counts=True))

# Convertir les étiquettes de cluster en int64
cluster_labels = db.labels_.astype('int64')

# Ajouter les labels de cluster à votre DataFrame en utilisant .loc
data_geo_blind.loc[:, 'cluster'] = cluster_labels

#
# Filtrer les données pour ne conserver que les points assignés à un cluster
data_geo_assigned_blind = data_geo_blind[data_geo_blind['cluster'] != -1]

# Définir le centre de la carte et le niveau de zoom pour afficher toutes les données
center_longitude = data_geo_assigned['merch_long'].mean()
center_latitude = data_geo_assigned['merch_lat'].mean()
zoom_level = 2.5

# Spécifier une palette de couleurs qualitative avec plus de variations
colorscale = px.colors.qualitative.Dark24

# Créer une carte interactive avec Plotly Express en spécifiant le centre et le zoom
fig_all_blind = px.scatter_mapbox(data_geo_assigned_blind, lon='merch_long', lat='merch_lat', color='cluster', 
                        mapbox_style="carto-positron", title='Clusters des zones de transactions avec DBSCAN en aveugle (sans la feature is_fraud)',
                        center=dict(lon=center_longitude, lat=center_latitude), zoom=zoom_level,
                        color_continuous_scale=colorscale)

# Afficher la carte interactive
fig_all_blind.show()

#Fig fraude par cluster

data_geo_assigned_blind['is_fraud'] = data_geo_copy['is_fraud']

# Calculer le nombre total de fraudes par cluster
fraud_count = data_geo_assigned_blind.groupby('cluster')['is_fraud'].sum()

# Calculer le pourcentage de fraudes pour chaque cluster
fraud_percentage = data_geo_assigned_blind.groupby('cluster')['is_fraud'].mean() * 100

# Créer un DataFrame à partir des résultats pour faciliter la visualisation avec Plotly Express
fraud_data = pd.DataFrame({'Cluster': fraud_count.index, 
                           'Nombre de fraudes': fraud_count.values,
                           'Pourcentage de fraudes': fraud_percentage.values})

# Trier les données par nombre de fraudes décroissant
fraud_data = fraud_data.sort_values(by='Nombre de fraudes', ascending=False)

# Créer le graphique à barres avec Plotly Express et spécifier la couleur en fonction du nombre de fraudes
fig = px.bar(fraud_data, x='Cluster', y='Pourcentage de fraudes', 
             title='Pourcentage de fraudes par cluster',
             labels={'Pourcentage de fraudes': 'Pourcentage de fraudes (%)'},
             color='Nombre de fraudes')

fig.show()

(array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
       dtype=int64),
 array([  43, 3532,   26,   86,  188,   46,   13,   32,   11,   18,   68,
          24,   11,   38,   25,   18,   14,   10,    6,    9,    5,    8,
          10,    9,    4,    5,    6,    9,    7,    3,    3,    3],
       dtype=int64))

C:\Users\mouto\AppData\Local\Temp\ipykernel_5348\104817110.py:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Comparaison entre les clusters de fraudes et les clusters de transactions

In [31]:
display(fig_fraud.show())
display(fig_all2.show())

None

None

La méthode DBSCAN sur l'ensemble des données nous fait remarquer que les transactions frauduleuses suivent la même logique que l'ensemble de données. Le DBSCAN sur les données globales arrive à retrouver similairement les mêmes clusters (soit le même groupe, soit par fusion de plusieurs clusters) même si on a plus de pertes par valeurs aberrantes.

### HDBSCAN

In [32]:
data_geo.head()

,merch_lat,merch_long,is_fraud
215462,34.149063,-87.536831,1
527279,42.053166,-105.675154,0
389352,42.354109,-80.667883,1
295584,33.191719,-116.946022,1
351609,39.618559,-75.336649,1


In [33]:
import hdbscan

# Paramètres de HDBSCAN
min_cluster_size = 3
min_samples = None  # Par défaut, il utilise min_cluster_size pour déterminer min_samples

# Création de l'instance HDBSCAN
hdb = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_samples, metric="euclidean")

# Entraînement du modèle HDBSCAN
hdb.fit(data_geo)

# Affichage des étiquettes de cluster et de leur compte
display(np.unique(hdb.labels_, return_counts=True))

# Créer une copie du DataFrame si nécessaire
data_geo_copy = data_geo.copy()

# Convertir les étiquettes de cluster en int64
cluster_labels = hdb.labels_.astype('int64')

# Ajouter les labels de cluster à votre DataFrame en utilisant .loc
data_geo_copy.loc[:, 'cluster'] = cluster_labels

# Filtrer les données pour ne conserver que les points assignés à un cluster
data_geo_assigned = data_geo_copy[data_geo_copy['cluster'] != -1]

# Définir le centre de la carte et le niveau de zoom pour afficher toutes les données
center_longitude = data_geo_assigned['merch_long'].mean()
center_latitude = data_geo_assigned['merch_lat'].mean()
zoom_level = 2

# Spécifier une palette de couleurs qualitative avec plus de variations
colorscale = px.colors.qualitative.Dark24

# Créer une carte interactive avec Plotly Express en spécifiant le centre et le zoom
fig_all_hdb = px.scatter_mapbox(data_geo_assigned, lon='merch_long', lat='merch_lat', color='cluster', 
                                mapbox_style="carto-positron", title='Clusters des zones de transactions avec HDBSCAN',
                                center=dict(lon=center_longitude, lat=center_latitude), zoom=zoom_level,
                                color_continuous_scale=colorscale)

# Afficher la carte interactive
fig_all_hdb.show()

# Calculer le nombre total de fraudes par cluster
fraud_count = data_geo_assigned.groupby('cluster')['is_fraud'].sum()

# Calculer le pourcentage de fraudes pour chaque cluster
fraud_percentage = data_geo_assigned.groupby('cluster')['is_fraud'].mean() * 100

# Créer un DataFrame à partir des résultats pour faciliter la visualisation avec Plotly Express
fraud_data_hdb = pd.DataFrame({'Cluster': fraud_count.index, 
                               'Nombre de fraudes': fraud_count.values,
                               'Pourcentage de fraudes': fraud_percentage.values})

# Trier les données par nombre de fraudes décroissant
fraud_data_hdb = fraud_data_hdb.sort_values(by='Nombre de fraudes', ascending=False)

# Créer le graphique à barres avec Plotly Express et spécifier la couleur en fonction du nombre de fraudes
fig_hdb = px.bar(fraud_data_hdb, x='Cluster', y='Pourcentage de fraudes', 
                 title='Pourcentage de fraudes par cluster (HDBSCAN)',
                 labels={'Pourcentage de fraudes': 'Pourcentage de fraudes (%)'},
                 color='Nombre de fraudes')

# Afficher le graphique
fig_hdb.show()


(array([ -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,
         12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,
         25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,
         38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,
         51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,
         64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
         77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,
         90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
        103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115,
        116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128,
        129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
        142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154,
        155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        168, 169, 170, 171, 172, 173, 174, 175, 176

In [34]:
# Filtrer les clusters avec au moins une fraude
clusters_with_frauds = fraud_data_hdb[fraud_data_hdb['Nombre de fraudes'] > 0]
fraudulent_clusters = clusters_with_frauds['Cluster'].unique()

# Filtrer les clusters sans fraude
clusters_without_frauds = fraud_data_hdb[fraud_data_hdb['Nombre de fraudes'] == 0]
non_fraudulent_clusters = clusters_without_frauds['Cluster'].unique()

# Créer une carte interactive pour les clusters avec au moins une fraude
fig_fraudulent_clusters = px.scatter_mapbox(data_geo_assigned[data_geo_assigned['cluster'].isin(fraudulent_clusters)], 
                                            lon='merch_long', lat='merch_lat', color='cluster', 
                                            mapbox_style="carto-positron", 
                                            title='Clusters avec au moins une fraude',
                                            center=dict(lon=center_longitude, lat=center_latitude), 
                                            zoom=zoom_level,
                                            color_continuous_scale=colorscale)
fig_fraudulent_clusters.show()

# Créer une carte interactive pour les clusters sans fraude
fig_non_fraudulent_clusters = px.scatter_mapbox(data_geo_assigned[data_geo_assigned['cluster'].isin(non_fraudulent_clusters)], 
                                                lon='merch_long', lat='merch_lat', color='cluster', 
                                                mapbox_style="carto-positron", 
                                                title='Clusters sans fraude',
                                                center=dict(lon=center_longitude, lat=center_latitude), 
                                                zoom=zoom_level,
                                                color_continuous_scale=colorscale)
fig_non_fraudulent_clusters.show()


### blind hdbscan

In [35]:
blind_geo = data_geo.copy()
blind_geo = blind_geo.drop(columns="is_fraud")

In [49]:
import hdbscan

# Paramètres de HDBSCAN
min_cluster_size = 3
min_samples = None  # Par défaut, il utilise min_cluster_size pour déterminer min_samples

# Création de l'instance HDBSCAN
hdb = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_samples, metric="euclidean")

# Entraînement du modèle HDBSCAN
hdb.fit(blind_geo)

# Affichage des étiquettes de cluster et de leur compte
display(np.unique(hdb.labels_, return_counts=True))

# Créer une copie du DataFrame si nécessaire
data_geo_copy = data_geo.copy()

# Convertir les étiquettes de cluster en int64
cluster_labels = hdb.labels_.astype('int64')

# Ajouter les labels de cluster à votre DataFrame en utilisant .loc
data_geo_copy.loc[:, 'cluster'] = cluster_labels

# Filtrer les données pour ne conserver que les points assignés à un cluster
data_geo_assigned = data_geo_copy[data_geo_copy['cluster'] != -1]

# Définir le centre de la carte et le niveau de zoom pour afficher toutes les données
center_longitude = data_geo_assigned['merch_long'].mean()
center_latitude = data_geo_assigned['merch_lat'].mean()
zoom_level = 2

# Spécifier une palette de couleurs qualitative avec plus de variations
colorscale = px.colors.qualitative.Dark24

# Créer une carte interactive avec Plotly Express en spécifiant le centre et le zoom
fig_all_hdb = px.scatter_mapbox(data_geo_assigned, lon='merch_long', lat='merch_lat', color='cluster', 
                                mapbox_style="carto-positron", title='Clusters des zones de transactions avec HDBSCAN',
                                center=dict(lon=center_longitude, lat=center_latitude), zoom=zoom_level,
                                color_continuous_scale=colorscale)

# Afficher la carte interactive
fig_all_hdb.show()

# Calculer le nombre total de fraudes par cluster
fraud_count = data_geo_assigned.groupby('cluster')['is_fraud'].sum()

# Calculer le pourcentage de fraudes pour chaque cluster
fraud_percentage = data_geo_assigned.groupby('cluster')['is_fraud'].mean() * 100

# Créer un DataFrame à partir des résultats pour faciliter la visualisation avec Plotly Express
fraud_data_hdb = pd.DataFrame({'Cluster': fraud_count.index, 
                               'Nombre de fraudes': fraud_count.values,
                               'Pourcentage de fraudes': fraud_percentage.values})

# Trier les données par nombre de fraudes décroissant
fraud_data_hdb = fraud_data_hdb.sort_values(by='Nombre de fraudes', ascending=False)

# Créer le graphique à barres avec Plotly Express et spécifier la couleur en fonction du nombre de fraudes
fig_hdb = px.bar(fraud_data_hdb, x='Cluster', y='Pourcentage de fraudes', 
                 title='Pourcentage de fraudes par cluster (HDBSCAN)',
                 labels={'Pourcentage de fraudes': 'Pourcentage de fraudes (%)'},
                 color='Nombre de fraudes')

# Afficher le graphique
fig_hdb.show()

# Calculer le nombre total de points par cluster
point_count = data_geo_assigned['cluster'].value_counts()

# Calculer le pourcentage de fraudes pour chaque cluster
fraud_percentage = data_geo_assigned.groupby('cluster')['is_fraud'].mean() * 100

# Créer un DataFrame à partir des résultats pour faciliter la visualisation avec Plotly Express
point_data_hdb = pd.DataFrame({'Cluster': point_count.index, 
                               'Nombre de points': point_count.values,
                               'Pourcentage de fraudes': fraud_percentage.values})

# Trier les données par nombre de points décroissant
point_data_hdb = point_data_hdb.sort_values(by='Nombre de points', ascending=False)

# Créer le graphique à barres avec Plotly Express et spécifier la couleur en fonction du pourcentage de fraudes
fig_point_hdb = px.bar(point_data_hdb, x='Cluster', y='Nombre de points', 
                       title='Nombre de points par cluster (HDBSCAN)',
                       labels={'Nombre de points': 'Nombre de points'},
                       color='Pourcentage de fraudes')

# Afficher le graphique
fig_point_hdb.show()


(array([ -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,
         12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,
         25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,
         38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,
         51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,
         64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
         77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,
         90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
        103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115,
        116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128,
        129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
        142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154,
        155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        168, 169, 170, 171, 172, 173, 174, 175, 176

### Preprocessing

In [37]:
data_geo_hdb = pd.DataFrame({
    'merch_long': clean_data['merch_long'],
    'merch_lat': clean_data['merch_lat'],
    'is_fraud': clean_data['is_fraud'],
    'category': clean_data['category']
})
data_geo_hdb.head()

data_geo_hdb_copy = data_geo_hdb.copy()

In [38]:
from sklearn.pipeline import Pipeline
categorical_features = ['category', 'is_fraud']
numerical_features = ['merch_lat', 'merch_long']
features = categorical_features + numerical_features

categorical_transformer = Pipeline(
    steps=[("encoder", OneHotEncoder(drop="first"))]
)

numeric_transformer = Pipeline(
    steps=[("scaler", StandardScaler())]
)

preprocessor = ColumnTransformer([
    ('categorical', categorical_transformer, categorical_features),
    ('numerical', numeric_transformer, numerical_features)
])

In [39]:
prepro_data_geo = preprocessor.fit_transform(data_geo_hdb)


In [40]:
type(prepro_data_geo)

scipy.sparse._csr.csr_matrix

In [41]:
import hdbscan

# Paramètres de HDBSCAN
min_cluster_size = 3
min_samples = None  # Par défaut, il utilise min_cluster_size pour déterminer min_samples

# Création de l'instance HDBSCAN
hdb = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_samples, metric="manhattan")

# Entraînement du modèle HDBSCAN
pred_hdb = hdb.fit_predict(prepro_data_geo)

# Affichage des étiquettes de cluster et de leur compte
display(np.unique(hdb.labels_, return_counts=True))

"""# Convertir la matrice creuse en DataFrame
prepro_data_geo_df = pd.DataFrame.sparse.from_spmatrix(prepro_data_geo)"""

# Ajouter les labels de cluster
data_geo_hdb['cluster'] = pred_hdb

"""
# Ajouter les labels de cluster à votre DataFrame en utilisant .loc
prepro_data_geo['cluster'] = pred_hdb
"""

(array([ -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,
         12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,
         25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,
         38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,
         51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,
         64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
         77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,
         90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
        103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115,
        116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128,
        129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
        142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154,
        155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        168, 169, 170, 171, 172, 173, 174, 175, 176

"\n# Ajouter les labels de cluster à votre DataFrame en utilisant .loc\nprepro_data_geo['cluster'] = pred_hdb\n"

In [42]:
# Filtrer les données pour ne conserver que les points assignés à un cluster
data_geo_assigned = data_geo_hdb[data_geo_hdb['cluster'] != -1]

# Définir le centre de la carte et le niveau de zoom pour afficher toutes les données
center_longitude = data_geo_assigned['merch_long'].mean()
center_latitude = data_geo_assigned['merch_lat'].mean()
zoom_level = 2

# Spécifier une palette de couleurs qualitative avec plus de variations
colorscale = px.colors.qualitative.Dark24

# Créer une carte interactive avec Plotly Express en spécifiant le centre et le zoom
fig_all_hdb = px.scatter_mapbox(data_geo_assigned, lon='merch_long', lat='merch_lat', color='cluster', 
                                mapbox_style="carto-positron", title='Clusters des zones de transactions avec HDBSCAN',
                                center=dict(lon=center_longitude, lat=center_latitude), zoom=zoom_level,
                                color_continuous_scale=colorscale)

# Afficher la carte interactive
fig_all_hdb.show()

In [43]:
# Calculer le nombre total de fraudes par cluster
fraud_count = data_geo_assigned.groupby('cluster')['is_fraud'].sum()

# Calculer le pourcentage de fraudes pour chaque cluster
fraud_percentage = data_geo_assigned.groupby('cluster')['is_fraud'].mean() * 100

# Créer un DataFrame à partir des résultats pour faciliter la visualisation avec Plotly Express
fraud_data_hdb = pd.DataFrame({'Cluster': fraud_count.index, 
                               'Nombre de fraudes': fraud_count.values,
                               'Pourcentage de fraudes': fraud_percentage.values})

# Trier les données par nombre de fraudes décroissant
fraud_data_hdb = fraud_data_hdb.sort_values(by='Nombre de fraudes', ascending=False)

# Créer le graphique à barres avec Plotly Express et spécifier la couleur en fonction du nombre de fraudes
fig_hdb = px.bar(fraud_data_hdb, x='Cluster', y='Pourcentage de fraudes', 
                 title='Pourcentage de fraudes par cluster (HDBSCAN)',
                 labels={'Pourcentage de fraudes': 'Pourcentage de fraudes (%)'},
                 color='Nombre de fraudes')

# Afficher le graphique
fig_hdb.show()


In [44]:
data_geo_hdb_copy['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'], format='%d/%m/%Y %H:%M')

data_geo_hdb_copy["day"]=data_geo_hdb_copy["trans_date_trans_time"].dt.day


colonne_a_retirer = ['trans_date_trans_time']
data_geo_hdb_copy = data_geo_hdb_copy.drop(colonne_a_retirer, axis=1)

data_geo_hdb_copy.head()

,merch_long,merch_lat,is_fraud,category,day
215462,-87.536831,34.149063,1,shopping_net,5
527279,-105.675154,42.053166,0,shopping_net,25
389352,-80.667883,42.354109,1,misc_net,20
295584,-116.946022,33.191719,1,personal_care,10
351609,-75.336649,39.618559,1,shopping_pos,3


In [45]:
prepro_data_geo2 = preprocessor.fit_transform(data_geo_hdb_copy)


In [46]:
import hdbscan

# Paramètres de HDBSCAN
min_cluster_size = 3
min_samples = None  # Par défaut, il utilise min_cluster_size pour déterminer min_samples

# Création de l'instance HDBSCAN
hdb = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_samples, metric="manhattan")

# Entraînement du modèle HDBSCAN
pred_hdb = hdb.fit_predict(prepro_data_geo2)

# Affichage des étiquettes de cluster et de leur compte
display(np.unique(hdb.labels_, return_counts=True))

"""# Convertir la matrice creuse en DataFrame
prepro_data_geo_df = pd.DataFrame.sparse.from_spmatrix(prepro_data_geo)"""

# Ajouter les labels de cluster
data_geo_hdb_copy['cluster'] = pred_hdb

"""
# Ajouter les labels de cluster à votre DataFrame en utilisant .loc
prepro_data_geo['cluster'] = pred_hdb
"""

(array([ -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,
         12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,
         25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,
         38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,
         51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,
         64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
         77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,
         90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
        103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115,
        116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128,
        129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
        142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154,
        155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        168, 169, 170, 171, 172, 173, 174, 175, 176

"\n# Ajouter les labels de cluster à votre DataFrame en utilisant .loc\nprepro_data_geo['cluster'] = pred_hdb\n"

In [47]:
# Filtrer les données pour ne conserver que les points assignés à un cluster
data_geo_assigned = data_geo_hdb[data_geo_hdb['cluster'] != -1]

# Définir le centre de la carte et le niveau de zoom pour afficher toutes les données
center_longitude = data_geo_assigned['merch_long'].mean()
center_latitude = data_geo_assigned['merch_lat'].mean()
zoom_level = 2

# Spécifier une palette de couleurs qualitative avec plus de variations
colorscale = px.colors.qualitative.Dark24

# Créer une carte interactive avec Plotly Express en spécifiant le centre et le zoom
fig_all_hdb = px.scatter_mapbox(data_geo_assigned, lon='merch_long', lat='merch_lat', color='cluster', 
                                mapbox_style="carto-positron", title='Clusters des zones de transactions avec HDBSCAN',
                                center=dict(lon=center_longitude, lat=center_latitude), zoom=zoom_level,
                                color_continuous_scale=colorscale)

# Afficher la carte interactive
fig_all_hdb.show()

In [48]:
# Calculer le nombre total de fraudes par cluster
fraud_count = data_geo_assigned.groupby('cluster')['is_fraud'].sum()

# Calculer le pourcentage de fraudes pour chaque cluster
fraud_percentage = data_geo_assigned.groupby('cluster')['is_fraud'].mean() * 100

# Créer un DataFrame à partir des résultats pour faciliter la visualisation avec Plotly Express
fraud_data_hdb = pd.DataFrame({'Cluster': fraud_count.index, 
                               'Nombre de fraudes': fraud_count.values,
                               'Pourcentage de fraudes': fraud_percentage.values})

# Trier les données par nombre de fraudes décroissant
fraud_data_hdb = fraud_data_hdb.sort_values(by='Nombre de fraudes', ascending=False)

# Créer le graphique à barres avec Plotly Express et spécifier la couleur en fonction du nombre de fraudes
fig_hdb = px.bar(fraud_data_hdb, x='Cluster', y='Pourcentage de fraudes', 
                 title='Pourcentage de fraudes par cluster (HDBSCAN)',
                 labels={'Pourcentage de fraudes': 'Pourcentage de fraudes (%)'},
                 color='Nombre de fraudes')

# Afficher le graphique
fig_hdb.show()
